In [1]:
# %matplotlib inline
import numpy as np
from sklearn.covariance import GraphicalLasso
import os
import sys
import math
import numpy as np
import pandas as pd
import nibabel as nib
from scipy import stats
from nilearn.maskers import NiftiLabelsMasker
from nilearn import plotting
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.cluster import KMeans
import importlib


import gi_gmn
import gi_gmn.utils as giu
import gi_gmn.models as gim

In [2]:
NP_train = 20
NC_train = 20
NP = 29
NC = 29
NPRE = 16
NP_test = NP - NP_train
NC_test = NC - NC_train
TR_P = 225
TR = 340
NRecord = 1
ROI = 120

#based on AAL atlas
all_col = ['Precentral_L', 'Precentral_R', 'Frontal_Sup_2_L', 'Frontal_Sup_2_R', 'Frontal_Mid_2_L', 'Frontal_Mid_2_R', 'Frontal_Inf_Oper_L', 'Frontal_Inf_Oper_R', 'Frontal_Inf_Tri_L', 'Frontal_Inf_Tri_R', 'Frontal_Inf_Orb_2_L', 'Frontal_Inf_Orb_2_R', 'Rolandic_Oper_L', 'Rolandic_Oper_R', 'Supp_Motor_Area_L', 'Supp_Motor_Area_R', 'Olfactory_L', 'Olfactory_R', 'Frontal_Sup_Medial_L', 'Frontal_Sup_Medial_R', 'Frontal_Med_Orb_L', 'Frontal_Med_Orb_R', 'Rectus_L', 'Rectus_R', 'OFCmed_L', 'OFCmed_R', 'OFCant_L', 'OFCant_R', 'OFCpost_L', 'OFCpost_R', 'OFClat_L', 'OFClat_R', 'Insula_L', 'Insula_R', 'Cingulate_Ant_L', 'Cingulate_Ant_R', 'Cingulate_Mid_L', 'Cingulate_Mid_R', 'Cingulate_Post_L', 'Cingulate_Post_R', 'Hippocampus_L', 'Hippocampus_R', 'ParaHippocampal_L', 'ParaHippocampal_R', 'Amygdala_L', 'Amygdala_R', 'Calcarine_L', 'Calcarine_R', 'Cuneus_L', 'Cuneus_R', 'Lingual_L', 'Lingual_R', 'Occipital_Sup_L', 'Occipital_Sup_R', 'Occipital_Mid_L', 'Occipital_Mid_R', 'Occipital_Inf_L', 'Occipital_Inf_R', 'Fusiform_L', 'Fusiform_R', 'Postcentral_L', 'Postcentral_R', 'Parietal_Sup_L', 'Parietal_Sup_R', 'Parietal_Inf_L', 'Parietal_Inf_R', 'SupraMarginal_L', 'SupraMarginal_R', 'Angular_L', 'Angular_R', 'Precuneus_L', 'Precuneus_R', 'Paracentral_Lobule_L', 'Paracentral_Lobule_R', 'Caudate_L', 'Caudate_R', 'Putamen_L', 'Putamen_R', 'Pallidum_L', 'Pallidum_R', 'Thalamus_L', 'Thalamus_R', 'Heschl_L', 'Heschl_R', 'Temporal_Sup_L', 'Temporal_Sup_R', 'Temporal_Pole_Sup_L', 'Temporal_Pole_Sup_R', 'Temporal_Mid_L', 'Temporal_Mid_R', 'Temporal_Pole_Mid_L', 'Temporal_Pole_Mid_R', 'Temporal_Inf_L', 'Temporal_Inf_R', 'Cerebelum_Crus1_L', 'Cerebelum_Crus1_R', 'Cerebelum_Crus2_L', 'Cerebelum_Crus2_R', 'Cerebelum_3_L', 'Cerebelum_3_R', 'Cerebelum_4_5_L', 'Cerebelum_4_5_R', 'Cerebelum_6_L', 'Cerebelum_6_R', 'Cerebelum_7b_L', 'Cerebelum_7b_R', 'Cerebelum_8_L', 'Cerebelum_8_R', 'Cerebelum_9_L', 'Cerebelum_9_R', 'Cerebelum_10_L', 'Cerebelum_10_R', 'Vermis_1_2', 'Vermis_3', 'Vermis_4_5', 'Vermis_6', 'Vermis_7', 'Vermis_8', 'Vermis_9', 'Vermis_10']


In [3]:
def get_subject_(path1,masker):  # two record version
        file_GM = nib.load(path1)             
        file_data = nib.load(path1).get_fdata()

        fileo = nib.Nifti1Image(file_data, file_GM.affine)  
        DMN_signals = masker.fit_transform(fileo)
        # print(DMN_signals.shape,NR)
        
        #Normalize time series of each ROI independetly with mean of one and std of zero
        for i in range(ROI):
            mu = np.mean(DMN_signals[:,i],axis=0)
            std = np.std(DMN_signals[:,i],axis=0)
            DMN_signals[:,i] = (DMN_signals[:,i] - mu)/std
        
        return DMN_signals

In [ ]:


path = "/users/grad/fmoradi/Graphical_LASSO/ADHD_Peking"

cnt =0
group1 = pd.DataFrame()

masker = NiftiLabelsMasker(labels_img="AAL_gm.nii", resampling_target = "data",standardize=True)

files = os.listdir(path)
for file in files:
    if file != ".DS_Store":
        print(file)
        group1_dir = path+"/"+file + "/" + "ses-1/func/" + file +"_ses-1_task-rest_run-1_bold.denoise/Denoised_"+ file +"_ses-1_task-rest_run-1_bold_MNI.nii.gz"
    #    /users/grad/fmoradi/Graphical_LASSO/ADHD_Peking/sub-1056121/ses-1/func/sub-1056121_ses-1_task-rest_run-1_bold.denoise/Denoised_sub-1056121_ses-1_task-rest_run-1_bold_MNI.nii.gz
        
        gr1 = get_subject_(group1_dir,masker)[:TR,:ROI]
        print(gr1.shape) 
        group1 = group1.append(pd.DataFrame(gr1),ignore_index=True)
  

In [ ]:
# group1.shape
pd.DataFrame(group1).to_csv( '../../ADHD_Peking/ADHD_peking.csv')
# !pwd


In [3]:
#reading data loading thm into matrix size of N*TR * ROI
control_data  = pd.read_csv('../../Ataxia/control_data_twoyear_last.csv',index_col=0).to_numpy()[:NC*TR,:] # limiting number of controls
patient_data = pd.read_csv('../../ADHD_Peking/ADHD_peking.csv',index_col=0).to_numpy()
patient_data.shape, control_data.shape


((6525, 120), (9860, 120))

In [4]:
importlib.reload(gi_gmn)
importlib.reload(gi_gmn.utils)
importlib.reload(gi_gmn.models)

# group level GMN - performance on test and saving preciison matrix 

alpha_p = 0.07
alpha_c = 0.07

model = gim.GroupSubjectGMN(
            NC, 
            NP, 
            NC_test, 
            TR,
            NP_test, 
            NRecord, 
            ROI
)
precision_p, precision_c = model.group_level(patient_data, control_data, alpha_p, alpha_c, True, TR_P)
precision_c.shape



Model: Group-level GMN
0.888888888888889 0.07027283689263066 0.9111111111111111 0.08314794192830981


(120, 120)

In [ ]:
# have to change tr +  in two types and unbalaaced data + feature  eng + mark + 

In [13]:
thr_weak = 0.1
Thresholding_diff = 0.11

copy_norm_p, copy_norm_c = model.prune_connections(precision_p, precision_c, thr_weak, Thresholding_diff)

giu.degree_sparsity(copy_norm_c), giu.degree_sparsity(copy_norm_p)

df_CONN, regions, connU = model.extract_connections_regions(copy_norm_p, copy_norm_c, all_col)
df_CONN.shape, regions.shape

((75, 4), (89,))

In [14]:

df_CONN, regions.shape
pd.DataFrame(regions).to_csv('../../ADHD_Peking/ROI40.csv',index = False) #496 * 42 
df_CONN.to_csv('../../ADHD_Peking/CONN61.csv',index = False) #496 * 42 

In [15]:
df_CONN.shape, regions.shape

((75, 4), (89,))

In [16]:
# first_model
alpha_p = 0.3
alpha_c = 0.3

all_pre_p, all_pre_c = model.individual_level(patient_data, control_data, alpha_p, alpha_c, regions, True, TR_P)

print(giu.degree_sparsity(all_pre_c[0,:]),giu.degree_sparsity(all_pre_p[0,:]))

(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)
(89, 89)


/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/sklearn/covariance/_graph_lasso.py:318: ConvergenceWarning: graphical_lasso: did not converge after 360 iteration: dual gap: -1.464e-03
  ConvergenceWarning,
/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/sklearn/covariance/_graph_lasso.py:318: ConvergenceWarning: graphical_lasso: did not converge after 360 iteration: dual gap: -2.560e-04
  ConvergenceWarning,


3.633128390974187e-05 3.9918753595623024e-05


In [17]:
#convert roiU andn connU to the new all_col array
masked_conn_patient, masked_conn_control = model.mask_CONN( all_pre_p, all_pre_c, all_col, regions, connU)

pd.DataFrame(masked_conn_control).to_csv('../../ADHD_Peking/Individual_precision_control.csv',index = False) #496*36 
pd.DataFrame(masked_conn_patient).to_csv('../../ADHD_Peking/Individual_precision_patient.csv',index = False) #496 * 42 

In [18]:
# np.count_nonzero(masked_conn_patient[7,:]),masked_conn_patient[1].shape np.CLIPnp
np.count_nonzero(masked_conn_patient), masked_conn_patient.shape

(1306, (29, 75))

In [19]:
masked_conn_control = pd.read_csv('../../ADHD_Peking/Individual_precision_control.csv').to_numpy()
masked_conn_patient = pd.read_csv('../../ADHD_Peking/Individual_precision_patient.csv').to_numpy()

model.binary_classfier(masked_conn_patient, masked_conn_control, "test", 1)

svmrbf
20 20 29 29
0.8722222222222221 0.9666666666666666 0.9649999999999999 0.9974999999999999 0.9194444444444443
0.10107447443700139 0.06186404847588914 0.03570714214271427 0.010897247358851692 0.04803226796052939
lr
20 20 29 29
0.8222222222222223 0.9888888888888889 0.9875 1.0 0.9055555555555556
0.08164965809277258 0.033333333333333354 0.021650635094610987 0.0 0.04339027597725918
svmlinear
20 20 29 29
0.95 0.9777777777777779 1.0 1.0 0.9638888888888888
0.07432826755699808 0.04444444444444446 0.0 0.0 0.04400827643820796
knn
20 20 29 29
0.7722222222222221 0.9 0.8625 0.9800000000000001 0.8361111111111112
0.113719386058104 0.07777777777777778 0.0649519052838329 0.0244948974278318 0.05407756203869938
rf
20 20 29 29
0.8666666666666666 0.85 1.0 1.0 0.8583333333333332
0.12472191289246472 0.11262852740396273 0.0 0.0 0.06448992647913275


# Feature Engineering

In [18]:

conn = pd.read_csv('../../ADHD_Peking/CONN.csv').to_numpy()
regions = pd.read_csv('../../ADHD_Peking/ROI.csv').to_numpy()
conn.shape

(42, 4)

In [20]:
from scipy import stats

#statistical test on selected 50 connections 
pen = conn.shape[0]

#statistical test 
thresold = 0.05 / pen 

for i in range(pen):
    # t, p = stats.stats.mannwhitneyu(avr_p[/i],avr_c[i])#deg_c[i,:],deg_p[i,:])
    t, p = stats.ttest_ind(masked_conn_control[:,i],masked_conn_patient[:,i])
    if p < thresold:
        print(conn[i][0:2])
        # print(masked_conn_control[:,i],masked_conn_patient[:,i])

 

['Frontal_Inf_Oper_L' 'Parietal_Inf_L']
['Olfactory_R' 'Rectus_R']
['Occipital_Mid_R' 'Temporal_Inf_L']
['OFCpost_R' 'OFClat_R']
['Frontal_Mid_2_R' 'Angular_R']
['Cerebelum_10_L' 'Cerebelum_10_R']
['Angular_R' 'Precuneus_R']
['Frontal_Sup_2_L' 'Supp_Motor_Area_L']
